# Augmentation using the Trained Convolutional Variational Autoencoder (CVAE)

### Yiming Jia</br> 2024-09-18
<br>

## User-defined parameters
### (same as P1_CVAE.ipynb)

In [ ]:
# File directory (change the directory to the folder including your time series data)
file_dir = 'C:/Users/jiayi/Desktop/CVAE Codes for GitHub/'


# Latent space size (usually it is a small number e.g., <= 8)
latent_dim = 4


# Number of epochs
NUM_EPOCH = 50000


# Stop training when loss reaching
Loss_Stop = 0.0000


# Initial learning rate for Adam optimizer
Learning_Rate = 0.005


# Initial learning rate step decay parameters
Drop_Rate1 = 0.80
NUM_EPOCH_DROP1 = 5000

Drop_Rate2 = 0.90
NUM_EPOCH_DROP2 = 10000

Drop_Rate3 = 0.95
NUM_EPOCH_DROP3 = 20000

Drop_Rate4 = 1.00


# Drop learning rate every Drop_Epoch
Drop_Epoch = 500


# Define a GPU usage
# CPU = -1, GPU0 = 0, GPU1 = 1
GPU_ID = -1

<br>

## Load packages and data
### (same as P1_CVAE.ipynb)

In [ ]:
# Check if the GPU is specified
import os
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID) 

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
# Create folders for model, data, and figure
import shutil

# Folder name
ID = f"_Lat{latent_dim}_Epoch{NUM_EPOCH}_Aug" 

# Model
model_dir = f"Model{str(ID)}" 
modelExist = os.path.exists(model_dir)
if modelExist:
    shutil.rmtree(model_dir)
    os.makedirs(model_dir) 
    print('Model folder exists, delete the current files in this folder.')
else:
    os.makedirs(model_dir)    
    print('Model folder does not exist, create this folder.')
    
# Data
data_dir = f"Data{str(ID)}" 
dataExist = os.path.exists(data_dir)
if dataExist:
    shutil.rmtree(data_dir)
    os.makedirs(data_dir) 
    print('Data folder exists, delete the current files in this folder.')
else:
    os.makedirs(data_dir)    
    print('Data folder does not exist, create this folder.')
        
# Figure
figure_dir = f"Figure{str(ID)}" 
figureExist = os.path.exists(figure_dir)
if figureExist:
    shutil.rmtree(figure_dir)    
    os.makedirs(figure_dir) 
    print('Figure folder exists, delete the current files in this folder.')
else:
    os.makedirs(figure_dir)    
    print('Figure folder does not exist, create this folder.')

In [ ]:
# Load packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Conv2DTranspose, Lambda, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
import scipy.io
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import time
from keras.layers import ReLU
from keras.layers import PReLU
from keras.layers import LeakyReLU

In [ ]:
# Load the time series data
Data = scipy.io.loadmat(file_dir + 'TimeSeries.mat')
# Notes: 
# 'TimeSeries.mat' is a Nt*1 vector of cells (Nt is the number of time series).
# Each cell includes a Ns*D matrix (Ni is the number of time steps, D is the number of dimensions).

T_Raw = Data["TimeSeries"]     

# Reshape the time series data
T_Raw = np.squeeze(T_Raw)
T = np.empty((T_Raw.shape[0], T_Raw[0].shape[0], T_Raw[0].shape[1]))
for i in range(T_Raw.shape[0]):
    T[i] = T_Raw[i]

T = T.reshape(-1, T.shape[1],T.shape[2], 1)

print('')
print('Time series data shape = ', T.shape)
print('')
print('Number of time series =', T.shape[0])
print('Number of time steps in each time series =', T.shape[1])
print('Number of dimensions in each time series = ', T.shape[2])

<br>

## Encoder
### (same as P1_CVAE.ipynb)

In [ ]:
# Input layer
input_encoder = Input(shape=(T.shape[1],T.shape[2],T.shape[3]), name = 'x_T')


# Activation function
act_fun = LeakyReLU(alpha=0.5)
# act_fun = 'tanh'
# act_fun = 'ReLU'
# act_fun = 'PReLU'


# Convolutional and max pooling layers
x_C1 = Conv2D(filters = 16, kernel_size = (3,3), padding="same",
              activation=act_fun, name = 'x_C1',
              kernel_initializer=initializers.GlorotNormal,
              bias_initializer=initializers.Zeros())(input_encoder)

x_P1 = MaxPooling2D(pool_size = (3,1), strides = (2,1), padding='same', name = 'x_P1')(x_C1)

x_C2 = Conv2D(filters = 8, kernel_size = (3,3), padding="same", 
              activation=act_fun, name = 'x_C2',
              kernel_initializer=initializers.GlorotNormal,
              bias_initializer=initializers.Zeros())(x_P1)

x_P2 = MaxPooling2D(pool_size = (3,1), strides = (2,1), padding='same', name = 'x_P2')(x_C2)

x_C3 = Conv2D(filters = 4, kernel_size = (3,3), padding="same",
              activation=act_fun, name = 'x_C3',
              kernel_initializer=initializers.GlorotNormal,
              bias_initializer=initializers.Zeros())(x_P2)


# Flatten layer
x_F = Flatten(name = 'x_F')(x_C3)


# Dense layers
x_D1 = Dense(512, activation=act_fun, name = 'x_D1',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(x_F)
x_D2 = Dense(128, activation=act_fun, name = 'x_D2',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(x_D1)
x_D3 = Dense(32, activation=act_fun, name = 'x_D3',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(x_D2)
x_D4 = Dense(8, activation=act_fun, name = 'x_D4',
                 kernel_initializer=initializers.GlorotNormal,
                 bias_initializer=initializers.Zeros())(x_D3)


# Latent feature layer
# Mean
encoder_mu = Dense(latent_dim, name="Encoder_Mu")(x_D4)
# Log-variance
encoder_log_variance = Dense(latent_dim, name="Encoder_LogVariance")(x_D4)


# Sampling
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = tensorflow.keras.backend.random_normal(shape=tensorflow.keras.backend.shape(mu), mean=0.0, stddev=1.0)
    random_sample = mu + tensorflow.keras.backend.exp(log_variance)*epsilon
    return random_sample


# Encoder
output_encoder = Lambda(sampling, name="Encoder_Output")([encoder_mu, encoder_log_variance])
encoder = Model([input_encoder], [output_encoder], name='encoder')

<br>

## Decoder 
### (same as P1_CVAE.ipynb)

In [ ]:
# Input layer
input_decoder = Input(shape=(latent_dim), name = 'Decoder_Input')


# Denses layer
y_D1 = Dense(8, activation=act_fun, name = 'y_D1',
                kernel_initializer=initializers.GlorotNormal,
                bias_initializer=initializers.Zeros())(input_decoder)
y_D2 = Dense(32, activation=act_fun, name = 'y_D2',
                 kernel_initializer=initializers.GlorotNormal,
                 bias_initializer=initializers.Zeros())(y_D1)
y_D3 = Dense(128, activation=act_fun, name = 'y_D3',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(y_D2)
y_D4 = Dense(512, activation=act_fun, name = 'y_D4',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(y_D3)
y_D5 = Dense(x_F.shape[1], activation=act_fun, name = 'y_D5',
                           kernel_initializer=initializers.GlorotNormal,
                           bias_initializer=initializers.Zeros())(y_D4)


# Inverse flatten layer
y_F = Reshape([x_C3.shape[1], x_C3.shape[2],x_C3.shape[3]], name = 'y_F')(y_D5)


# Deconvolutional layers
y_C1 = Conv2DTranspose(filters = 8, strides = (1,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C1',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_F)                                                     

y_C2 = Conv2DTranspose(filters = 8, strides = (2,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C2',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C1)                                                     

y_C3 = Conv2DTranspose(filters = 16, strides = (1,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C3',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C2)  

y_C4 = Conv2DTranspose(filters = 16, strides = (2,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C4',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C3)   

y_C5 = Conv2DTranspose(filters = 1, kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C5',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C4)    

output_decoder = Conv2DTranspose(filters = 1, kernel_size = (3,3), padding="same", 
                                 activation='linear', name = 'Decorder_Output',
                                 kernel_initializer=initializers.GlorotNormal,
                                 bias_initializer=initializers.Zeros())(y_C5) 


# Decoder
decoder = Model([input_decoder], [output_decoder], name='decoder')

<br>

## Autoencoder 
### (same as P1_CVAE.ipynb)

In [ ]:
# Input
input_auto = input_encoder

# Output
output_auto = decoder(encoder(input_encoder))

# Autoencoder
autoencoder = Model([input_auto], [output_auto], name='auto')

<br>

## Training parameters
### (same as P1_CVAE.ipynb)

In [ ]:
# Checkpoint ID
ID_CVAE = f"_Lat{latent_dim}_Epoch{NUM_EPOCH}" 
model_auto_dir = f"Model{str(ID_CVAE)}" 
Checkpoint_ID = os.path.join(model_auto_dir, "Checkpoint.ckpt")

<br>

## Load the trained autoencoder and decoder

In [ ]:
# Autoencoder
autoencoder_best = autoencoder

# Load weights
autoencoder_best.load_weights(Checkpoint_ID);

In [ ]:
# Decoder
decoder_best = decoder

# Load weights from autoencoder_best
encoder_len = len(encoder.weights)
decoder_len = len(decoder_best.weights)
decoder_best.set_weights(autoencoder_best.weights[encoder_len:encoder_len+decoder_len])

<br> 

## Augmentation

In [ ]:
start = time.time()

# Load the latent features generated for augmentation
Aug = scipy.io.loadmat(file_dir + 'Aug_Lat.mat')
    
Random = Aug["Lat_Ran"]     

print('Encoder augmented data shape:', Random.shape)
print('')
    
    
# Generate the augmentated data using the decorder
decoder_aug_raw = decoder_best.predict(Random,verbose = 0);
decoder_aug = decoder_aug_raw.reshape(Random.shape[0],T.shape[1]*T.shape[2]);
np.savetxt(data_dir + '/Aug.dat', decoder_aug);
    
end = time.time()

In [ ]:
print("The running time:", round((end-start)/60,1), "mins")
print('')